### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")


### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
# filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [4]:
# Create an empty list to store the reviews
reviews_list = []
# loop through pages 0-19
page = 0
while page <= 2:
    # create query with a page number
    # API results show 10 articles at a time
    # Make a "GET" request and retrieve the JSON
    updated_url_with_page = f"{query_url}&page={page}"
    response = requests.get(updated_url_with_page)
    reviews = response.json()
    print(updated_url_with_page)
    print(reviews)
    print(json.dumps(reviews, indent=4))
    

    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list

    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for doc in reviews["response"]["docs"]:
            reviews_list.append(doc)
        # Print the page that was just retrieved
        print(f"Found articles for the page {page}")
    except:
        # Print the page number that had no results then break from the loop
        print(f"No articles found in the page {page}")
    page = page + 1



https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=67FjEmLsdtmlBbCYWzpop5UCDPGlw0Vt&begin_date=20130101&end_date=20230531&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count&page=0
{'status': 'OK', 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html', 'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.', 'source': 'The New York Times', 'headline': {'main': '‘The Attachment Diaries’ Review: Love, Sick', 'kicker': None, 'content_kicker': None, 'print_headline': 'The Attachment Diaries', 'name': None, 'seo': None, 'sub': None}, 'keywords': [{'name': 'subject', 'value': 'Movies', 'rank': 1, 'major': 'N'}, {'name': 'creative_works', 'value': 'The Attachment Di

In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
for reviews in reviews_list:
    print(json.dumps(reviews, indent=4))


{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
            "rank": 3,
            "major": "N"
        }
    ],
  

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head(4)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None


In [7]:
title_list = []
for reviews in reviews_list:
    # Extract the title from the "headline.main" column and
    title = reviews["headline"]["main"]
    start = title.index('\u2018')
    end = title.index('\u2019',start+1)
    # Title is between unicode characters \u2018 and \u2019. 
    substring_tittle = title[start+1:end]
    # End string should include " Review" to avoid cutting title early
    # title = substring_tittle + " Review"
    title_list.append(substring_tittle)
# save it to a new column "title"
reviews_df["title"] = title_list
reviews_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People


In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df.loc[:, "keywords"].astype(str)
for reviews in reviews_list:
    keywords_list = []
    keywords_list = reviews["keywords"]
    extract_keyword = extract_keywords(keywords_list)
    reviews_df["keywords"] = extract_keyword
reviews_df.head(3)


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever


In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
tittle = reviews_df["title"]
tittle_list = tittle.to_list()
tittle_list

['The Attachment Diaries',
 'What',
 'You Can Live Forever',
 'A Tourist',
 'Other People',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love']

### Access The Movie Database API

In [10]:
# Prepare The Movie Database query
tmdb_url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key


In [11]:
# Create an empty list to store the results
tmdb_movies_list = []
movies_data = {}
tmdb_movie_detail_url = "https://api.themoviedb.org/3/movie/"

# Create a request counter to sleep the requests after a multiple
# of 50 requests


# Loop through the titles
for mov_tit in tittle_list:
    movie_data = {}

    # Check if we need to sleep before making a request

    # Add 1 to the request counter

    
    # Perform a "GET" request for The Movie Database
    #print(mov_tit)
    #print(tmdb_url + mov_tit + tmdb_key_string)
    response_data = requests.get(tmdb_url + mov_tit + tmdb_key_string)
    mov_title_data = response_data.json()
    movie_data['title'] = mov_tit
    
    # print(json.dumps(mov_title_data, indent=4))


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

    try:
        # loop through the movie tittle ["results"] to get the movie id
        for doc in mov_title_data["results"]:
            movie_id = doc['id']
            genre_list = []
            spoken_languages_list = []
            production_countries_list = []
            #print(tmdb_movie_detail_url + str(doc['id']) + "?" + tmdb_key_string)
            # Get movie id
            # Make a request for a the full movie details
            # Execute "GET" request with url
            tmdb_movie_detail_by_id = requests.get(tmdb_movie_detail_url + str(doc['id']) + "?" + tmdb_key_string)
            tmdb_movie_detail = tmdb_movie_detail_by_id.json()
            #print(json.dumps(tmdb_movie_detail, indent=4))
            # Add the relevant data to a dictionary and
            # Extract the genre names into a list
            for genre in tmdb_movie_detail["genres"]:
                genre_list.append(genre['name'])
            # print(f"genre_list {genre_list}")
            movie_data['genres'] = genre_list
            # Extract the spoken_languages' English name into a list
            for spoken_languages in tmdb_movie_detail["spoken_languages"]:
                spoken_languages_list.append(spoken_languages['english_name'])
            #print(f"spoken_languages_list {spoken_languages_list}")
            movie_data['spoken_languages'] = spoken_languages_list
            # Extract the production_countries' name into a list
            for production_countries in tmdb_movie_detail["production_countries"]:
                production_countries_list.append(production_countries['name'])
            #print(f"production_countries_list {production_countries_list}")
            movie_data['production_countries'] = production_countries_list
            #print(f"Movie dictionry:", movie_data)
            # append it to the tmdb_movies_list list 
            tmdb_movies_list.append(movie_data)
        # Print the page that was just retrieved
        print(f"Movie Found :", mov_tit)
    except:
        # Print the page number that had no results then break from the loop
        print(f"Movie Ids Not Found")

# Print out the title that was found
print(f"Final Movie dictionry:", tmdb_movies_list)


Movie Found : The Attachment Diaries
Movie Found : What
Movie Found : You Can Live Forever
Movie Found : A Tourist
Movie Found : Other People
Movie Found : One True Loves
Movie Found : The Lost Weekend: A Love Story
Movie Found : A Thousand and One
Movie Found : Your Place or Mine
Movie Found : Love in the Time of Fentanyl
Movie Found : Pamela, a Love Story
Movie Found : In From the Side
Movie Found : After Love
Movie Found : Alcarràs
Movie Found : Nelly & Nadine
Movie Found : Lady Chatterley
Movie Found : The Sound of Christmas
Movie Found : The Inspection
Movie Found : Bones and All
Movie Found : My Policeman
Movie Found : About Fate
Movie Found : Waiting for Bojangles
Movie Found : I Love My Dad
Movie Found : A Love Song
Movie Found : Alone Together
Movie Found : Art of Love
Movie Found : The Wheel
Movie Found : Thor: Love and Thunder
Movie Found : Both Sides of the Blade
Movie Found : Fire of Love
Final Movie dictionry: [{'title': 'The Attachment Diaries', 'genres': ['Drama', 'Myst

In [12]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
json.dumps(tmdb_movies_list, indent=4)

'[\n    {\n        "title": "The Attachment Diaries",\n        "genres": [\n            "Drama",\n            "Mystery",\n            "Thriller",\n            "Horror"\n        ],\n        "spoken_languages": [\n            "Spanish"\n        ],\n        "production_countries": [\n            "Argentina"\n        ]\n    },\n    {\n        "title": "What",\n        "genres": [\n            "Comedy",\n            "Drama"\n        ],\n        "spoken_languages": [\n            "German"\n        ],\n        "production_countries": [\n            "Germany"\n        ]\n    },\n    {\n        "title": "What",\n        "genres": [\n            "Comedy",\n            "Drama"\n        ],\n        "spoken_languages": [\n            "German"\n        ],\n        "production_countries": [\n            "Germany"\n        ]\n    },\n    {\n        "title": "What",\n        "genres": [\n            "Comedy",\n            "Drama"\n        ],\n        "spoken_languages": [\n            "German"\n       

In [13]:
# Convert the results to a DataFrame
all_results_df = pd.json_normalize(tmdb_movies_list)
all_results_df.head(5)


,title,genres,spoken_languages,production_countries
0,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,What,"[Comedy, Drama]",[German],[Germany]
2,What,"[Comedy, Drama]",[German],[Germany]
3,What,"[Comedy, Drama]",[German],[Germany]
4,What,"[Comedy, Drama]",[German],[Germany]


### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title
merge_df = pd.merge(reviews_df, all_results_df, on="title", how="outer")
merge_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What,"[Comedy, Drama]",[German],[Germany]
2,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What,"[Comedy, Drama]",[German],[Germany]
3,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What,"[Comedy, Drama]",[German],[Germany]
4,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What,"[Comedy, Drama]",[German],[Germany]


In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
#merge_df_copy = merge_df.copy()
#merge_df_copy["spoken_languages"] = merge_df_copy.loc[:, "spoken_languages"].astype(str)
#merge_df['spoken_languages'] = merge_df['spoken_languages'].str.strip('[]')
#merge_df.head(5)
#merge_df_copy.dtypes
#merge_df['spoken_languages'] = merge_df['spoken_languages'].astype('str')
#merge_df['spoken_languages'] = merge_df['spoken_languages'].str.replace(r"\[","")
#merge_df['spoken_languages'].apply(lambda x:x.replace('[', '').replace(']', ''))

merge_df['spoken_languages'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in merge_df['spoken_languages']])
merge_df['genres'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in merge_df['genres']])
merge_df['production_countries'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in merge_df['production_countries']])

merge_df.head(1)
# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina


In [16]:
# Drop "byline.person" column
#merge_df.drop(columns=['byline.person'])
del merge_df['byline.person']
merge_df.head(5)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization,title,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
2,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
3,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany
4,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,subject: Documentary Films and Programs;subjec...,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None,What,"Comedy, Drama",German,Germany


In [20]:
# Delete duplicate rows and reset index
merge_df_reinded_df = merge_df["title"].drop_duplicates().reset_index(drop=True)
merge_df_reinded_df

0             The Attachment Diaries
1                               What
2               You Can Live Forever
3                          A Tourist
4                       Other People
5                     One True Loves
6     The Lost Weekend: A Love Story
7                 A Thousand and One
8                 Your Place or Mine
9       Love in the Time of Fentanyl
10              Pamela, a Love Story
11                  In From the Side
12                        After Love
13                          Alcarràs
14                    Nelly & Nadine
15                   Lady Chatterley
16            The Sound of Christmas
17                    The Inspection
18                     Bones and All
19                      My Policeman
20                        About Fate
21             Waiting for Bojangles
22                     I Love My Dad
23                       A Love Song
24                    Alone Together
25                       Art of Love
26                         The Wheel
2

In [18]:
# Export data to CSV without the index
merge_df.to_csv("movies_list.csv", index=False)